<a href="https://colab.research.google.com/github/biamouras/alura_pyspark_challenge/blob/main/Notebooks/02_tratamento_dados_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Insightplaces - Tratamento dos dados para ML

A Insightplaces, plataforma fantasia de imóveis da Alura, está com dificuldades para definir os valores do imóveis e gostaria de fazer de modo automatizado. Ainda, a empresa percebeu que o recomendador atual não está sendo efetivo para gerar novos cliques entre os usuários.

Nosso projeto será trabalhar com os dados disponibilizados de imóveis no Rio de Janeiro para ajudar a empresa a lidar com estes dois problemas usando regressão para a definição dos valores e a análise de agrupamentos através de Machine Learning. 

Este segundo projeto tem o foco de preparar os dados para os modelos de previsão.